# Import Packages

# Introduction
<br>
In this tutorial, we will look at a simple linear ODE system:

$$
\frac{du}{dt} = a u + b v \\
\frac{dv}{dt} = c u + d v  
$$

with the boundary conditions:

$$
u(t=0) = u_0 \\
v(t=0) = v_0
$$

Given the solutions $u(t)$ and $v(t)$, we want to use BayesFlow to predict the parameters $a$, $b$, $c$ and $d$ of the ODE equations as well as the boundary conditions $u_0$ and $v_0$.  


# Analytical Solution

The advantage